In [32]:
import pandas as pd
import numpy as np 
import string

#checks if a given word is of english language
def isEnglishLetter(s):
    try:
        s.encode('ascii')
        return 1
    except:
        return 0

#reads input csv file    
data = pd.read_csv("out.csv")

In [33]:
##preprocessing

#remove all rows with field of study and industry are undefined
#since those 2 information are the descriptive information for the study
data = data[(data['educations-field_of_study'] != "--undefined--") & (data['industry'] != "--undefined--")]

#gets the significant columns only
columns = ["industry", "educations-degree", "educations-field_of_study", "educations-school-name", "positions-is-current", "num-connections"]
new_data = data[columns]

#changes all work status from undefined to False (assumption: if undefined then the person is not working)
new_data['positions-is-current'] = new_data['positions-is-current'].replace('--undefined--', 'FALSE')

#replace all undefined values to nan
new_data = new_data.replace("--undefined--",np.NaN)
new_data = new_data.reset_index()

#removes rows that are not english
is_englishLetter = []
for i in range(0, new_data.shape[0]):
    is_englishLetter.append(isEnglishLetter(new_data["industry"][i]))
new_data["is_english_letter"] = is_englishLetter
new_data = new_data[(new_data["is_english_letter"] == 1)]

#drop none english letters
del new_data["is_english_letter"]
new_data = new_data.reset_index()
del new_data["level_0"]
del new_data["index"]

#changes to UpperCase all values
for i in columns:
    new_data[i] = [str(j).upper() for j in new_data[i].tolist()]
new_data

/Users/zenitharnejo/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,industry,educations-degree,educations-field_of_study,educations-school-name,positions-is-current,num-connections
0,INFORMATION TECHNOLOGY AND SERVICES,BE,ELECTRONICS AND COMMUNICATION,THE MADRAS SEVA SADAN HIGHER SECONDARY SCHOOL,FALSE,1
1,EDUCATION MANAGEMENT,MASTER OF BUSINESS ADMINISTRATION (M.B.A.),MARKETING,MUMBAI UNIVERSITY,FALSE,13
2,EDUCATION MANAGEMENT,ISLAMIC EDUCATION,ISLAMIC EDUCATION,UNIVERSITI MALAYA,TRUE,0
3,PUBLIC RELATIONS AND COMMUNICATIONS,MASTER'S,PUBLIC COMMUNICATION,INSTITUT D'ETUDES POLITIQUES DE LILLE,FALSE,112
4,AUTOMOTIVE,MCA,COMPUTER APPLICATIONS,ST. JOSEPH COLLEGE CUDDALORE,FALSE,18
5,FINANCIAL SERVICES,PGDMM,MARKETING,ANNAMALAI UNIVERSITY,TRUE,10
6,COMPUTER SOFTWARE,B.TECH,INFORMATION TECHNOLOGY,ANNA UNIVERSITY,TRUE,56
7,ACCOUNTING,B.COM,COMMERCE,KERALA UNIVERSITY,FALSE,2
8,"LEISURE, TRAVEL & TOURISM",BSC,"PHYSICS,CHEMISTRY,MATHS",MANGALORE UNIVERSITY,FALSE,74
9,INFORMATION TECHNOLOGY AND SERVICES,MASTER OF TECHNOLOGY (M.TECH.),COMPUTER SCIENCE,VISVESVARAYA TECHNOLOGICAL UNIVERSITY,FALSE,75


In [34]:
import enchant
import re

d = enchant.Dict("en_US")
industry = new_data["industry"]#.unique()

english_word = []
for i in range(0, len(industry)):
    value = industry[i]
    res = re.findall(r"[\w']+", value)

    j = 0
    flag = False
    for j in range(0, len(res)):
        if d.check(res[j]) == False:
            english_word.append(False)
            flag = True
            break
    if flag == False:
        english_word.append(True)

new_data["english_word"] = english_word
new_data = new_data[(new_data["english_word"] == True)]

#drop none english words
del new_data["english_word"]
new_data = new_data.reset_index()
del new_data["index"]
new_data

,industry,educations-degree,educations-field_of_study,educations-school-name,positions-is-current,num-connections
0,INFORMATION TECHNOLOGY AND SERVICES,BE,ELECTRONICS AND COMMUNICATION,THE MADRAS SEVA SADAN HIGHER SECONDARY SCHOOL,FALSE,1
1,EDUCATION MANAGEMENT,MASTER OF BUSINESS ADMINISTRATION (M.B.A.),MARKETING,MUMBAI UNIVERSITY,FALSE,13
2,EDUCATION MANAGEMENT,ISLAMIC EDUCATION,ISLAMIC EDUCATION,UNIVERSITI MALAYA,TRUE,0
3,PUBLIC RELATIONS AND COMMUNICATIONS,MASTER'S,PUBLIC COMMUNICATION,INSTITUT D'ETUDES POLITIQUES DE LILLE,FALSE,112
4,AUTOMOTIVE,MCA,COMPUTER APPLICATIONS,ST. JOSEPH COLLEGE CUDDALORE,FALSE,18
5,FINANCIAL SERVICES,PGDMM,MARKETING,ANNAMALAI UNIVERSITY,TRUE,10
6,COMPUTER SOFTWARE,B.TECH,INFORMATION TECHNOLOGY,ANNA UNIVERSITY,TRUE,56
7,ACCOUNTING,B.COM,COMMERCE,KERALA UNIVERSITY,FALSE,2
8,"LEISURE, TRAVEL & TOURISM",BSC,"PHYSICS,CHEMISTRY,MATHS",MANGALORE UNIVERSITY,FALSE,74
9,INFORMATION TECHNOLOGY AND SERVICES,MASTER OF TECHNOLOGY (M.TECH.),COMPUTER SCIENCE,VISVESVARAYA TECHNOLOGICAL UNIVERSITY,FALSE,75


In [35]:
new_data["industry"].unique()


array(['INFORMATION TECHNOLOGY AND SERVICES', 'EDUCATION MANAGEMENT',
       'PUBLIC RELATIONS AND COMMUNICATIONS', 'AUTOMOTIVE',
       'FINANCIAL SERVICES', 'COMPUTER SOFTWARE', 'ACCOUNTING',
       'LEISURE, TRAVEL & TOURISM', 'HOSPITAL & HEALTH CARE', 'CHEMICALS',
       'BANKING', 'TRANSPORTATION/TRUCKING/RAILROAD', 'BIOTECHNOLOGY',
       'RETAIL', 'CONSTRUCTION', 'AIRLINES/AVIATION', 'HOSPITALITY',
       'MECHANICAL OR INDUSTRIAL ENGINEERING', 'POLITICAL ORGANIZATION',
       'TELECOMMUNICATIONS', 'LEGAL SERVICES', 'DESIGN', 'PHARMACEUTICALS',
       'SPORTS', 'HUMAN RESOURCES', 'INTERNET',
       'ELECTRICAL/ELECTRONIC MANUFACTURING', 'MARKETING AND ADVERTISING',
       'STAFFING AND RECRUITING', 'HEALTH, WELLNESS AND FITNESS',
       'MINING & METALS', 'NONPROFIT ORGANIZATION MANAGEMENT', 'INSURANCE',
       'BROADCAST MEDIA', 'HIGHER EDUCATION', 'ENVIRONMENTAL SERVICES',
       'WRITING AND EDITING', 'OIL & ENERGY', 'UTILITIES',
       'INDUSTRIAL AUTOMATION', 'MEDICAL DEVIC